In [2]:
!pip install spacy

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple


In [3]:
!pip install en_core_web_sm-3.1.0-py3-none-any.whl

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple
Processing ./en_core_web_sm-3.1.0-py3-none-any.whl
en-core-web-sm is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
import spacy
import en_core_web_sm

2023-03-07 19:20:27.160306: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
import pandas as pd

In [7]:
df_iva_sample = pd.read_csv("s3://adl-core-sagemaker-studio/external/Deepali/query202208-12.csv")

/tmp/ipykernel_19/1269742681.py:1: DtypeWarning: Columns (36,37,38,41,42,43,44,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iva_sample = pd.read_csv("s3://adl-core-sagemaker-studio/external/Deepali/query202208-12.csv")


In [8]:
df_iva_sample_1 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202201.csv")

/tmp/ipykernel_19/1666922987.py:1: DtypeWarning: Columns (36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iva_sample_1 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202201.csv")


In [9]:
df_iva_sample_2 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202202-05.csv")

/tmp/ipykernel_19/2423618519.py:1: DtypeWarning: Columns (36,38,41,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iva_sample_2 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202202-05.csv")


In [10]:
df_iva_sample_3 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202206.csv")

/tmp/ipykernel_19/3523077079.py:1: DtypeWarning: Columns (36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iva_sample_3 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202206.csv")


In [11]:
df_iva_sample_4 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202207.csv")

/tmp/ipykernel_19/3794833382.py:1: DtypeWarning: Columns (36,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iva_sample_4 = pd.read_csv("s3://adl-core-sagemaker-studio/external/IVA/query202207.csv")


In [12]:
df_iva = pd.concat([df_iva_sample, df_iva_sample_1, df_iva_sample_2, df_iva_sample_3, df_iva_sample_4 ], axis = 0)

In [13]:
df_iva = df_iva.drop_duplicates()

In [14]:
df_iva.shape

(11859121, 67)

In [15]:
df_iva = df_iva[['client_id','person_internal_id','input','unit_name','response_text','next_unit_hit','previous_unit_hit','knowledge_category']].drop_duplicates()

In [16]:
df_iva.shape

(9379811, 8)

In [17]:
def count_intnt_entits(text):
    doc = nlp(text)
    intents = [token.text for token in doc if token.pos_ == 'VERB']
    entities = [token.text for token in doc if token.pos_ in {'NOUN', 'PROPN', 'ADJ', 'NUM', 'ADV'}]
    return len(intents), len(entities)

import re
def clean_text(text):
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text

def extract_ner_entities(sentence):
    doc = nlp(sentence)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

import numpy as np
def length_entities(list_entities):
    if (list_entities==np.nan or list_entities==None or list_entities==''):
        return 0
    else:
        return len(list_entities)
    
filter_func = lambda x: any(item[1] in ["PERSON", "GPE", "ORG", "LOC", "FAC"] for item in x)

In [18]:
list_1 = ['ira','RMD','HRdirect','livechat','what is my hsa','P45','Payslip?',
    'sps','F80.2','ub','What is YSA','Paystub please','Sh','mfv','C-128','ax','no is hsa','FormL564','HIS','cif','GreT','YSACard',
    'Heli','RxPCN','403(b)','Hsa yes or no','ypr','Gv','ONA?','What is UHC?','HC-2','uo','what is 4DX?','osh','what is my hsa?',
    'sPRAVATO','sdr','RMD’s','coverage?How','This is for my hsa','pto?','A&DD','childcareplus','fs','mbi','Is that my lowesbenefit.com',
    'hra yes','mri?']
    
def text_preprocess(dataframe, list_1):
    dataframe = dataframe.drop_duplicates()
    dataframe = dataframe[dataframe['knowledge_category']!='Technical Units']
    dataframe['input_cleaned'] = dataframe.input.fillna('')
    dataframe[['no_of_intents', 'no_of_entities']] = dataframe.input_cleaned.apply(lambda x: pd.Series(count_intnt_entits(x)))  
    dataframe = dataframe[((dataframe['no_of_intents']>0) & (dataframe['no_of_entities']>0))|(dataframe['input_cleaned'].isin(list_1))]
    dataframe = dataframe.drop(['no_of_intents','no_of_entities'], axis=1)
    dataframe = dataframe.drop_duplicates(subset = ['input','input_cleaned','unit_name','response_text','next_unit_hit','previous_unit_hit','knowledge_category'])
    dataframe['input_cleaned'] = dataframe['input_cleaned'].apply(clean_text)
    dataframe['ner_enities'] = dataframe['input_cleaned'].apply(extract_ner_entities)
    dataframe['len_ner_enities'] = dataframe['ner_enities'].apply(length_entities)
    dataframe3 = dataframe[dataframe['len_ner_enities']>0]
    dataframe4 = dataframe[dataframe['len_ner_enities']==0]
    dataframe5 = dataframe3[~dataframe3["ner_enities"].apply(filter_func)]
    dataframe6 = pd.concat([dataframe4, dataframe5], axis = 0)
    dataframe6 = dataframe6.drop(['ner_enities','len_ner_enities'], axis=1)
    dataframe6['input_changed']=dataframe6['input']!=dataframe6['input_cleaned']
#     df_iva6.to_excel('IVA_all_cat_claened_data.xlsx')
    
    return dataframe6

In [ ]:
df_iva_cleaned = text_preprocess(df_iva, list_1)

In [ ]:
words_2 = ['day care', 'creche', 'child care', 
 'nursery', 'nursery school', 'preschool', 'after-school care', 'day nursery', 'infant school', 
 'infant care', 'playschool', 'play-group', 'kindergarten', 'childminding', 'babysitting', 'babysitter', 
 'nanny care', 'children supervision', 'toddler care']

df_iva_cleaned_cc = df_iva_cleaned[df_iva_cleaned['input'].str.contains('|'.join(words_2), case=False, na=False)][['client_id','person_internal_id','input','labels']]

In [ ]:
df_iva_cleaned_cc['category'] = 'Child care'

In [ ]:
df_cleaned_iva_non_child = df_iva_cleaned[~df_iva_cleaned['input'].str.contains('|'.join(words_2), case=False, na=False)][['client_id','person_internal_id','input','input_cleaned','next_unit_hit','unit_name','knowledge_category','response_text']]

In [ ]:
df_cleaned_iva_non_child_1 = df_cleaned_iva_non_child.head(df_iva_cleaned_cc.shape[0])

In [ ]:
df_cleaned_iva_non_child_1['category'] = 'other'

In [ ]:
df_cleaned_iva_child_non_child_care = pd.concat([df_iva_cleaned_cc, df_cleaned_iva_non_child_1], axis = 0)

In [ ]:
df_cleaned_iva_child_non_child_care_1 = df_cleaned_iva_child_non_child_care[['client_id','person_internal_id','input','input_cleaned','category','next_unit_hit','unit_name']]

In [ ]:
df_cleaned_iva_child_non_child_care_2 = df_cleaned_iva_child_non_child_care_1.sample(frac=1, random_state=42)

In [ ]:
df_cleaned_iva_child_non_child_care_2.loc[df_cleaned_iva_child_non_child_care_2['input_cleaned'].str.lower().str.contains('childelder'), 'category'] = 'Child/Elder care'

In [ ]:
df_cleaned_iva_child_non_child_care_2.to_excel('IVA_data_child_care_model.xlsx')

In [ ]:
df_cleaned_iva_child_non_child_care_3 = df_cleaned_iva_child_non_child_care_2.drop(['input','next_unit_hit','unit_name'], axis=1)

In [ ]:
df_cleaned_iva_child_non_child_care_3.head(5)

In [ ]:
df_cleaned_iva_child_non_child_care_3.shape

--------

--------

## Building Elder care model

-----

In [ ]:
import transformers
import torch
import numpy as np

import numpy as np
import regex as re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
import tokenizers
from transformers import RobertaTokenizer, TFRobertaModel

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

----

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
# from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the dataset
# Replace `X_train`, `Y_train`, `X_test`, and `Y_test` with your own data
X_train, X_test, Y_train, Y_test = train_test_split(df_cleaned_iva_child_non_child_care_3['input_cleaned'], df_cleaned_iva_child_non_child_care_3['category'], test_size=0.2)

In [ ]:
# Encode the labels
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)

In [ ]:
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#distilbert-base-uncased

# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
# Tokenize the input data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

In [ ]:
# Convert the input data into PyTorch tensors
X_train = torch.tensor(train_encodings['input_ids'])
train_masks = torch.tensor(train_encodings['attention_mask'])
Y_train = torch.tensor(Y_train)
X_test = torch.tensor(test_encodings['input_ids'])
test_masks = torch.tensor(test_encodings['attention_mask'])
Y_test = torch.tensor(Y_test)

In [ ]:
# Define the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(le.classes_))
# model = DistilBertModel.from_pretrained('distilbert-base-uncased', num_labels=len(le.classes_))

In [ ]:
# Set up the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 25

In [ ]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train.to(device), attention_mask=train_masks.to(device), labels=Y_train.to(device))
    loss = loss_fn(outputs[1], Y_train.to(device))
    loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test.to(device), attention_mask=test_masks.to(device))
        val_loss = loss_fn(val_outputs.logits, Y_test.to(device))
        print(f"Epoch {epoch}: train_loss={loss:.3f}, val_loss={val_loss:.3f}")


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Get the model's predictions for the test set
with torch.no_grad():
    test_outputs = model(X_test.to(device), attention_mask=test_masks.to(device))
    test_preds = np.argmax(test_outputs.logits.detach().cpu().numpy(), axis=1)

# Create the confusion matrix
confusion = confusion_matrix(Y_test, test_preds)

print("Confusion matrix:")
print(confusion)

In [ ]:
# Plot the confusion matrix as a heatmap
sns.heatmap(confusion, annot=True, cmap='Blues', fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.show()

In [ ]:
# Save the model
# 15 epochs model
# torch.save({
#             'epoch': epochs,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'train_loss': 0.098,
#             'val_loss': 0.041,
#             }, '15epoch_model.pth')

In [260]:
# Save the model
# 25 epochs model
# torch.save({
#             'epoch': epochs,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'train_loss': 0.005,
#             'val_loss': 0.010,
#             }, '25epoch_model.pth')

In [246]:
# Load the model
# checkpoint = torch.load('model.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# train_loss_history = checkpoint['train_loss']
# val_loss_history = checkpoint['val_loss']
# hyperparams = checkpoint['hyperparams']